In [15]:
# import os
# print(__file__)
# os.chdir(r'C:\\Users\\ashin\\Documents\\Dev\\velib_ds\src\\')
from velibdata import VelibData
from transform import VelibTransformer

In [16]:
from velibdata import VelibData

processed_df = VelibData(cache=True, update_cache=False).extract().transform().data

2025-08-17 17:26:13,204 - Transformation: velib_data
2025-08-17 17:26:17,845 - Suppression de 79606 (1.0%) doublons d'origine, c'est-à-dire des lignes identiques par les valeurs clé de données de temps réel Vélib: 'dt', 'bikes', 'capacity' et 'station'.
2025-08-17 17:26:23,421 - Il y a 100573 (1.27%) lignes qui réprésentent les mêmes stations pour les même dates et heures. On enlève les valeurs intrahoraires et recalcule les deltas.
2025-08-17 17:26:31,445 - Suppression de 0 valeurs manquantes.


KeyboardInterrupt: 

In [14]:
# Date minimale pour train
import datetime
from transform import VelibTransformerDefaults


LOW_LIMIT = datetime.date(2024, 12, 2)
# Date marge de train et test
# SPLIT_DATE = datetime.date(2025, 5, 19)
SPLIT_DATE = None # Dernière date - 7j
# Limite supérieur
CUT_OFF_DATE = None 
# ID de la station Chatelet
chatelet = '82328045'

transform_params = {
    'low_limit' : LOW_LIMIT,
    'split_date' : SPLIT_DATE or (datetime.datetime.now() - datetime.timedelta(days=7)).date(), # params ou J-7
    'cut_off_date' : CUT_OFF_DATE,
    'smoothen' : True, # Lissage activé
    'smooth_window' : 3, # taille de fenetre de lissage: 3, 5, 7...
    'weeklag' : True, # Ajouter un feature de delta lag - une semaine. Prédiction limité par une semaine en avance.
    'weeklagmean' : False, # Moyen de delta de cette station, cette heure, ce jour de semaine et ce flag de jour férier (pour 30j derniers).
    'weeklag4hours': True, # Moyen de fenetre de 4h de delta lag - une semaine.
    'peakmeans' : True, # Moyen d'heures de pointe pour 30 dernier jours pour cette station
    'reconstructed' : False, # Garder les données reconstruite ou non
    # 'hotencode' : ['cluster'],
    'hotencode' : [], # One hot encode features
    'encode_time' : VelibTransformerDefaults.params['encode_time'] + [('month', 12)], # features pour le sin-cos encoding
    'nonscale' : VelibTransformerDefaults.params['nonscale'] + ['cluster'] + ['station'] # features a ne pas normaliser
}

In [1]:
from general import DataFiles
import pandas as pd
data = pd.read_hdf(DataFiles.processed, key='df')
# data[['station', 'lat', 'lon', 'name', 'capacity']].drop_duplicates().to_hdf(DataFiles.processed_stations, key='df', mode='w')


In [3]:
import datetime
last_timestamp = data['datehour'].max()
cutoff_7days = last_timestamp - datetime.timedelta(days=7)
data[data['datehour'] >= cutoff_7days].to_hdf(DataFiles.processed_7d, key='df', mode='w')            


C:\Users\ashin\AppData\Local\Temp\ipykernel_3836\2851293386.py:4: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block1_values] [items->Index(['date', 'station', 'name'], dtype='object')]

  data[data['datehour'] >= cutoff_7days].to_hdf(DataFiles.processed_7d, key='df', mode='w')


In [12]:
import joblib
import pandas as pd
import numpy as np

def sum_or_nan(values : pd.Series):
    '''
    Summarize only sequences without NA. Else return NA.
    '''
    if values.isna().any():
        return np.nan
    else:
        return values.sum()


data = joblib.load('../test.joblib')
print('Loaded')
df = data.data
df['dayoff'] = df['holiday'] | df['pont']

aggregation_task = {
    'delta6h12h' : pd.NamedAgg(column='delta', aggfunc=lambda x: sum_or_nan(x.iloc[5:12])),
    'delta16h22h' : pd.NamedAgg(column='delta', aggfunc=lambda x: sum_or_nan(x.iloc[15:22])),
    'weekday' : pd.NamedAgg('weekday', 'first')
}


Loaded


In [ ]:
import pytz
import datetime
import pandas as pd
from transform import VelibTransformer, VelibTransformerDefaults
from velibdata import VelibData
import os

os.environ['LOKY_MAX_CPU_COUNT'] = '2'

LOW_LIMIT = datetime.datetime.combine(datetime.date(2024, 12, 2), datetime.time())
# CUT_OFF_DATE = None 
CUT_OFF_DATE = datetime.datetime(2025, 7, 22, 0, tzinfo=pytz.timezone('Europe/Paris'))
SPLIT_DATE = CUT_OFF_DATE - datetime.timedelta(days=7) # Dernière date - 7j
# SPLIT_DATE = None # Dernière date - 7j
chatelet = '82328045'

transform_params = {
    'low_limit' : LOW_LIMIT,
    'split_date' : SPLIT_DATE or datetime.datetime.combine(datetime.date.today(), datetime.time()) - datetime.timedelta(days=7), # params ou J-7
    'cut_off_date' : CUT_OFF_DATE,
    'smoothen' : True, # Lissage activé
    'smooth_window' : 3, # taille de fenetre de lissage: 3, 5, 7...
    'weeklag' : True, # Ajouter un feature de delta lag - une semaine. Prédiction limité par une semaine en avance.
    'weeklagmean' : False, # Moyen de delta de cette station, cette heure, ce jour de semaine et ce flag de jour férier (pour 30j derniers).
    'weeklag4hours': True, # Moyen de fenetre de 4h de delta lag - une semaine.
    'peakmeans' : True, # Moyen d'heures de pointe pour 30 dernier jours pour cette station
    'reconstructed' : False, # Garder les données reconstruite ou non
    'hotencode' : ['cluster'],
    'encode_time' : VelibTransformerDefaults.params['encode_time'] + [('month', 12)], # features pour le sin-cos encoding
    # 'nonscale' : VelibTransformerDefaults.params['nonscale'] + ['cluster'] + ['station'] # features a ne pas normaliser
}

# data = VelibData(cache=False, update_cache=True, from_dt=LOW_LIMIT, to_dt=CUT_OFF_DATE)
data = VelibData(cache=True, from_dt=LOW_LIMIT, to_dt=CUT_OFF_DATE)
data.extract().transform()


2025-08-17 17:39:13,043 - Transformation: velib_data
2025-08-17 17:39:16,863 - Suppression de 79606 (1.0%) doublons d'origine, c'est-à-dire des lignes identiques par les valeurs clé de données de temps réel Vélib: 'dt', 'bikes', 'capacity' et 'station'.
2025-08-17 17:39:22,669 - Il y a 100573 (1.27%) lignes qui réprésentent les mêmes stations pour les même dates et heures. On enlève les valeurs intrahoraires et recalcule les deltas.
2025-08-17 17:39:30,553 - Suppression de 0 valeurs manquantes.
2025-08-17 17:39:32,197 - Reconstruction de saisonalité.
2025-08-17 17:39:48,697 - Valeur min-max: 21 - 5434
2025-08-17 17:39:48,705 - Seuils de outliers: 5173.0 - 5523.0
2025-08-17 17:39:48,705 - Nombre de valeurs total: 1465
2025-08-17 17:39:48,705 - Grands outliers: 0 ou 0.0%
2025-08-17 17:39:48,711 - Petits outliers: 79 ou 5.39%
2025-08-17 17:39:48,714 - On enlève 79 stations peu représentées dans le dataset.
2025-08-17 17:39:50,003 - On a supprimé 11114 lignes, 0.14%.
2025-08-17 17:40:34,16

ValueError: The truth value of a DataFrame is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [7]:
# df = VelibTransformer.smoothen(data.data)
df = data.data.copy()
df, df_train, df_test = VelibTransformer.split(df, transform_params)
df_test_orig = df_test.copy()
transformer = VelibTransformer(transform_params)
df_train = transformer.fit_transform(df_train)
df_test = transformer.transform(df_test)

2025-08-17 17:55:47,800 - Splitting data: 2024-12-02 00:00:00 - 2025-07-15 00:00:00+00:09 - 2025-07-22 00:00:00+00:09
2025-08-17 17:55:58,471 - Split completed.
2025-08-17 17:56:00,636 - Fitting dataset with shape: (7840680, 24)
2025-08-17 17:56:00,636 - Fitting VelibClusterizer...
2025-08-17 17:56:01,385 - Calculating stations statistics for clusterizing
2025-08-17 17:56:10,781 - Converting lat and lon to km position
2025-08-17 17:56:10,792 - Calculating best silhouette score for clusters
2025-08-17 17:56:10,793 - Calculating silhouet score for clusters 0 of 100...
2025-08-17 17:56:10,854 - Calculating silhouet score for clusters 1 of 100...
2025-08-17 17:56:10,915 - Calculating silhouet score for clusters 2 of 100...
2025-08-17 17:56:10,975 - Calculating silhouet score for clusters 3 of 100...
2025-08-17 17:56:11,038 - Calculating silhouet score for clusters 4 of 100...
2025-08-17 17:56:11,100 - Calculating silhouet score for clusters 5 of 100...
2025-08-17 17:56:11,164 - Calculating

In [22]:
def get_mare(df_test : pd.DataFrame):
    '''Get Mean Absolute Rush Error i.e. MAE for sums of flow at 6-12 and 15-22 per day-station'''
    df = df_test[df_test.hour.between(6, 12) | df_test.hour.between(15, 22)].copy()
    df['morning'] = df.hour <= 12
    # Count total flow for morning and evening rush hours per day-station 
    scoring = df.groupby(['station', 'date', 'morning'], as_index=False)[['delta_test', 'delta_pred']].sum()
    # Get absolute error for rush hours
    scoring['are'] = (scoring['delta_test'] - scoring['delta_pred']).abs()
    metrics = {}
    metrics['mare'] = float(scoring['are'].mean())
    metrics['chatelet_mare'] = float(scoring[scoring.station.astype(str) == chatelet]['are'].mean())
    test_date = df_test[df_test.weekday == 3].datehour.dt.date.iloc[0]
    metrics['date_mare'] = float(scoring[scoring.date == test_date]['are'].mean())
    metrics['date_chatelet_mare'] = float(scoring[(scoring.date == test_date) & (scoring.station.astype(str) == chatelet)]['are'].mean())
    return metrics    

In [26]:
from sklearn.linear_model import BayesianRidge
from general import DataFiles, load_transformer_params
from transform import VelibTransformer
import pandas as pd
import datetime

df = pd.read_hdf(DataFiles.processed, key='df')
transformer_params = load_transformer_params()
if transformer_params.get('smoothen'):
    df = VelibTransformer.smoothen(df, transformer_params.get('smooth_window', 3))

df, df_train, df_test = VelibTransformer.split(df,
{'split_date' : datetime.datetime.combine(datetime.date.today(), datetime.time()) - datetime.timedelta(days=14)}
)
# Fit and save VelibTransformer
transformer = VelibTransformer(transformer_params)
df_train = transformer.fit_transform(df_train)
df_test = transformer.transform(df_test)



2025-08-23 13:48:45,277 - Smoothening data window=3...
2025-08-23 13:48:51,508 - Data smoothened.
2025-08-23 13:48:51,804 - Splitting data: None - 2025-08-09 00:00:00 - None
2025-08-23 13:48:58,410 - Split completed.
2025-08-23 13:49:00,845 - Fitting dataset with shape: (8130436, 24)
2025-08-23 13:49:00,845 - Fitting VelibClusterizer...
2025-08-23 13:49:01,690 - Calculating stations statistics for clusterizing
2025-08-23 13:49:12,152 - Converting lat and lon to km position
2025-08-23 13:49:12,159 - Calculating best silhouette score for clusters
2025-08-23 13:49:12,162 - Calculating silhouet score for clusters 0 of 100...
2025-08-23 13:49:12,255 - Calculating silhouet score for clusters 1 of 100...
2025-08-23 13:49:12,349 - Calculating silhouet score for clusters 2 of 100...
2025-08-23 13:49:12,421 - Calculating silhouet score for clusters 3 of 100...
2025-08-23 13:49:12,498 - Calculating silhouet score for clusters 4 of 100...
2025-08-23 13:49:12,582 - Calculating silhouet score for cl

ValueError: The feature names should match those that were passed during fit.
Feature names must be in the same order as they were in fit.


In [39]:
df_test = df_test.iloc[:, :-4]

In [40]:
columns = [c for c in df_train.columns if c not in ['datehour', 'station', 'cluster', 'metacluster']]

model = BayesianRidge()
model.fit(df_train[columns].drop(columns='delta'), df_train['delta'])
y_pred = model.predict(df_test[columns].drop(columns='delta'))
df_test_metrics = df_test.dropna().copy()
df_test_metrics['delta_test'] = df_test_metrics['delta']
df_test_metrics['delta_pred'] = y_pred
chatelet = '82328045'
df_test_metrics['date'] = df_test_metrics['datehour'].dt.date
df_test_metrics['hour'] = df_test_metrics['datehour'].dt.hour
df_test_metrics['weekday'] = df_test_metrics['datehour'].dt.weekday
metrics = get_mare(df_test_metrics)
print(metrics)


{'mare': 5.307975447454164, 'chatelet_mare': 3.664814814814749, 'date_mare': 2.3333333333334614, 'date_chatelet_mare': nan}


In [41]:
pd.set_option("display.float_format", "{:.6f}".format)

print(df_test_metrics[['delta_test', 'delta_pred']])

         delta_test  delta_pred
8146068    3.666667   -0.333333
8147534   -3.666667    0.333333
8163518    0.000000    0.333333
8164984    0.000000    0.000000
8193913   -0.111111    1.666667
...             ...         ...
8473475    3.500000    1.000000
8473476    0.000000   -1.000000
8473477   -0.500000   -2.666667
8473478   -2.500000    0.000000
8473479    1.500000    3.000000

[246580 rows x 2 columns]


In [ ]:
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(max_depth=10, n_estimators=50, min_samples_split=5, min_samples_leaf=5)
model.fit(df_train[columns].drop(columns='delta'), df_train['delta'])
y_pred = model.predict(df_test[columns].drop(columns='delta'))
df_test_metrics = df_test[(~df_test.reconstructed)].dropna().copy()
df_test_metrics['delta_test'] = df_test_metrics['delta']
df_test_metrics['delta_pred'] = y_pred
chatelet = '82328045'
df_test_metrics['date'] = pd.to_datetime(df_test_metrics['date'])
metrics = get_mare(df_test_metrics)
print(metrics)

{'mare': 10.354559889433416, 'chatelet_mare': 17.785714285714285, 'date_mare': 12.321057428720827, 'date_chatelet_mare': 22.0}


In [ ]:
import os
os.environ['LOKY_MAX_CPU_COUNT'] = '2'


EXPECTED_COLUMNS = list(sorted(['datehour', 'date', 'station', 'lat', 'lon', 'month', 'name', 'hour', 'weekday',
       'weekend', 'holiday', 'preholiday', 'postholiday', 'pont', 'vacances',
       'vacances_uni', 'reconstructed', 'capacity', 'bikes', 'delta', 'temp',
       'precip', 'gel', 'vent']))

In [6]:
import pytz
import datetime
import pandas as pd
from transform import VelibTransformer, VelibTransformerDefaults
from velibdata import VelibData


LOW_LIMIT = datetime.datetime.combine(datetime.date(2024, 12, 2), datetime.time())
# CUT_OFF_DATE = None 
CUT_OFF_DATE = datetime.datetime(2025, 7, 22, 0, tzinfo=pytz.timezone('Europe/Paris'))
SPLIT_DATE = CUT_OFF_DATE - datetime.timedelta(days=7) # Dernière date - 7j
# SPLIT_DATE = None # Dernière date - 7j
chatelet = '82328045'

transform_params = {
    'low_limit' : LOW_LIMIT,
    'split_date' : SPLIT_DATE or datetime.datetime.combine(datetime.date.today(), datetime.time()) - datetime.timedelta(days=7), # params ou J-7
    'cut_off_date' : CUT_OFF_DATE,
    'smoothen' : True, # Lissage activé
    'smooth_window' : 3, # taille de fenetre de lissage: 3, 5, 7...
    'weeklag' : True, # Ajouter un feature de delta lag - une semaine. Prédiction limité par une semaine en avance.
    'weeklagmean' : False, # Moyen de delta de cette station, cette heure, ce jour de semaine et ce flag de jour férier (pour 30j derniers).
    'weeklag4hours': True, # Moyen de fenetre de 4h de delta lag - une semaine.
    'peakmeans' : True, # Moyen d'heures de pointe pour 30 dernier jours pour cette station
    'reconstructed' : False, # Garder les données reconstruite ou non
    'hotencode' : ['cluster'],
    'encode_time' : VelibTransformerDefaults.params['encode_time'] + [('month', 12)], # features pour le sin-cos encoding
    # 'nonscale' : VelibTransformerDefaults.params['nonscale'] + ['cluster'] + ['station'] # features a ne pas normaliser
}

# data = VelibData(cache=False, update_cache=True, from_dt=LOW_LIMIT, to_dt=CUT_OFF_DATE)
data = VelibData(cache=True, from_dt=LOW_LIMIT, to_dt=CUT_OFF_DATE)
data.extract().transform()
df, df_train, df_test = VelibTransformer.split(data.data, transform_params)
df_test_orig = df_test.copy()
transformer = VelibTransformer(transform_params)
df_train = transformer.fit_transform(df_train)
df_test = transformer.transform(df_test)

2025-08-17 16:29:06,378 - Transformation: velib_data
2025-08-17 16:29:10,144 - Suppression de 79606 (1.0%) doublons d'origine, c'est-à-dire des lignes identiques par les valeurs clé de données de temps réel Vélib: 'dt', 'bikes', 'capacity' et 'station'.
2025-08-17 16:29:15,772 - Il y a 100573 (1.27%) lignes qui réprésentent les mêmes stations pour les même dates et heures. On enlève les valeurs intrahoraires et recalcule les deltas.
2025-08-17 16:29:23,996 - Suppression de 0 valeurs manquantes.
2025-08-17 16:29:25,735 - Reconstruction de saisonalité.
2025-08-17 16:29:41,133 - Valeur min-max: 21 - 5434
2025-08-17 16:29:41,135 - Seuils de outliers: 5173.0 - 5523.0
2025-08-17 16:29:41,137 - Nombre de valeurs total: 1465
2025-08-17 16:29:41,140 - Grands outliers: 0 ou 0.0%
2025-08-17 16:29:41,140 - Petits outliers: 79 ou 5.39%
2025-08-17 16:29:41,140 - On enlève 79 stations peu représentées dans le dataset.
2025-08-17 16:29:42,332 - On a supprimé 11114 lignes, 0.14%.
2025-08-17 16:30:20,93

KeyboardInterrupt: 

Testing cases

In [2]:
from general import DataFiles, load_transformer_params
from transform import VelibTransformer
import pandas as pd
df = pd.read_hdf(DataFiles.processed, key='df')
transformer_params = load_transformer_params()
if transformer_params.get('smoothen'):
    df = VelibTransformer.smoothen(df, transformer_params.get('smooth_window', 3))
# Fit and save VelibTransformer
transformer = VelibTransformer(transformer_params)
df_transformed = transformer.fit_transform(df)
transformer.save(DataFiles.transformer)


2025-08-23 12:35:55,081 - Smoothening data window=3...
2025-08-23 12:36:01,148 - Data smoothened.
2025-08-23 12:36:02,174 - Fitting dataset with shape: (8473480, 24)
2025-08-23 12:36:02,175 - Fitting VelibClusterizer...
2025-08-23 12:36:02,933 - Calculating stations statistics for clusterizing
2025-08-23 12:36:06,313 - Converting lat and lon to km position
2025-08-23 12:36:06,319 - Calculating best silhouette score for clusters
2025-08-23 12:36:06,320 - Calculating silhouet score for clusters 0 of 100...
2025-08-23 12:36:06,381 - Calculating silhouet score for clusters 1 of 100...
2025-08-23 12:36:06,444 - Calculating silhouet score for clusters 2 of 100...
2025-08-23 12:36:06,505 - Calculating silhouet score for clusters 3 of 100...
2025-08-23 12:36:06,570 - Calculating silhouet score for clusters 4 of 100...
2025-08-23 12:36:06,636 - Calculating silhouet score for clusters 5 of 100...
2025-08-23 12:36:06,704 - Calculating silhouet score for clusters 6 of 100...
2025-08-23 12:36:06,76

In [3]:
df_transformed.to_hdf('test.h5', key='df', mode='w')

In [10]:
import dill
from general import Storage

# model = dill.load(open(Storage.models('best.joblib'), 'rb'))
dill.dump(['test'], 'test.pkl')

TypeError: file must have a 'write' attribute

In [13]:
dill.dump(model, open('test.pkl', 'wb'))

In [17]:
joblib.dump(model, 'test2.joblib')

['test2.joblib']

In [26]:
model_2 = dill.load(open('test.pkl', 'rb'))

In [24]:
model_3 = joblib.load('test2.joblib')


In [1]:
from mlpflow import MLPFlow
from general import load_best_params
import pandas as pd

df_transformed = pd.read_hdf('test.h5', key='df')

model = MLPFlow()
model.set_params(**load_best_params())
model.fit(df_transformed.drop(columns='delta'), df_transformed['delta'])
model.save('best')


2025-08-24 00:34:31,206 - Created instance for MLP_flow
2025-08-24 00:34:31,289 - Params set to: {'wl': 3.0, 'wh': 3.0, 'loss_fn': 'wmae', 'batch': 256000, 'opt': 'adam', 'emb': '', 'act': 'relu', 'epochs': 3}
c:\Users\ashin\Documents\Dev\velib_ds\.venv\Lib\site-packages\mlflow\types\utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(
2025-08-24 00:34

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ features (InputLayer)           │ (None, 23)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ concatenate (Concatenate)       │ (None, 23)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │         1,536 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 16)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 8)              │           136 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 1)              │             9 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,289 (16.75 KB)

 Trainable params: 4,289 (16.75 KB)

 Non-trainable params: 0 (0.00 B)

2025-08-24 00:34:32,593 - Training model with params: {'wl': 3.0, 'wh': 3.0, 'loss_fn': 'wmae', 'batch': 256000, 'opt': 'adam', 'emb': '', 'act': 'relu', 'epochs': 3}


Epoch 1/3
23/23 - 10s - 431ms/step - loss: 2.2067 - mae: 1.2041 - val_loss: 2.2292 - val_mae: 1.2195 - learning_rate: 1.0000e-03
Epoch 2/3
23/23 - 7s - 296ms/step - loss: 2.2003 - mae: 1.2004 - val_loss: 2.2183 - val_mae: 1.2142 - learning_rate: 1.0000e-03
Epoch 3/3
23/23 - 7s - 305ms/step - loss: 2.1901 - mae: 1.1963 - val_loss: 2.1983 - val_mae: 1.2051 - learning_rate: 1.0000e-03


2025-08-24 00:35:01,163 - Model compiled and fitted.
2025-08-24 00:35:01,639 - Saved model to c:\Users\ashin\Documents\Dev\velib_ds\local\models\best.pkl
2025-08-24 00:35:01,901 - Saved MLFlow run magnificent-yak-517
2025-08-24 00:35:01,903 - All saved.


🏃 View run magnificent-yak-517 at: http://127.0.0.1:8080/#/experiments/1/runs/5c66d8bbb07844118974ee1aeaf07087
🧪 View experiment at: http://127.0.0.1:8080/#/experiments/1


In [2]:
import dill
from general import Storage

test = dill.load(open(Storage.models('best' + '.pkl'), 'rb'))

In [3]:
print(test.__dict__)

{'name': 'MLP_flow', 'mlflow_uri': 'http://127.0.0.1:8080', 'metrics': {}, 'params': {'wl': 3.0, 'wh': 3.0, 'loss_fn': 'wmae', 'batch': 256000, 'opt': 'adam', 'emb': '', 'act': 'relu', 'epochs': 3}, 'model': None, 'input_example': None, 'baselib': 'keras', 'mode': 'test', 'model_filename': None, 'logger': <Logger dual_logger (DEBUG)>, 'X_train': None, 'df_test': None, 'X_test': None, 'y_pred': None, 'dataid': '20241204_23-20250822_17_train', 'embedding': {}, 'history': None, 'y_train': None, 'DELTA_TOP_MARGE': np.float64(2.6666666666666665), 'DELTA_BOTTOM_MARGE': np.float64(-2.6666666666666665)}


In [5]:
MLPFlow.load('best').model.predict(pd.DataFrame())

c:\Users\ashin\Documents\Dev\velib_ds\.venv\Lib\site-packages\keras\src\trainers\epoch_iterator.py:151: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


ValueError: math domain error

In [5]:
from mlpflow import MLPFlow
from general import load_best_params
import pandas as pd
df = pd.read_hdf('test.h5', key='df')
model = MLPFlow()
model.set_params(**load_best_params())
model.fit(df.drop(columns='delta'), df['delta'])
model.save('best')


2025-08-23 12:53:47,399 - Created instance for MLP_flow
2025-08-23 12:53:47,546 - Params set to: {'wl': 3.0, 'wh': 3.0, 'loss_fn': 'wmae', 'batch': 256000, 'opt': 'adam', 'emb': '', 'act': 'relu', 'epochs': 300}
c:\Users\ashin\Documents\Dev\velib_ds\.venv\Lib\site-packages\mlflow\types\utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(
2025-08-23 12:

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ features (InputLayer)           │ (None, 23)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ concatenate (Concatenate)       │ (None, 23)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │         1,536 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 16)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 8)              │           136 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 1)              │             9 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,289 (16.75 KB)

 Trainable params: 4,289 (16.75 KB)

 Non-trainable params: 0 (0.00 B)

2025-08-23 12:53:49,648 - Training model with params: {'wl': 3.0, 'wh': 3.0, 'loss_fn': 'wmae', 'batch': 256000, 'opt': 'adam', 'emb': '', 'act': 'relu', 'epochs': 300}


Epoch 1/300
23/23 - 12s - 511ms/step - loss: 2.2101 - mae: 1.2075 - val_loss: 2.2296 - val_mae: 1.2197 - learning_rate: 1.0000e-03
Epoch 2/300
23/23 - 7s - 319ms/step - loss: 2.2018 - mae: 1.2012 - val_loss: 2.2227 - val_mae: 1.2167 - learning_rate: 1.0000e-03
Epoch 3/300
23/23 - 8s - 337ms/step - loss: 2.1914 - mae: 1.1979 - val_loss: 2.1917 - val_mae: 1.2037 - learning_rate: 1.0000e-03
Epoch 4/300
23/23 - 7s - 314ms/step - loss: 2.1632 - mae: 1.1901 - val_loss: 2.1321 - val_mae: 1.1833 - learning_rate: 1.0000e-03
Epoch 5/300
23/23 - 7s - 314ms/step - loss: 2.1129 - mae: 1.1793 - val_loss: 2.0477 - val_mae: 1.1595 - learning_rate: 1.0000e-03
Epoch 6/300
23/23 - 7s - 307ms/step - loss: 2.0447 - mae: 1.1685 - val_loss: 1.9569 - val_mae: 1.1485 - learning_rate: 1.0000e-03
Epoch 7/300
23/23 - 7s - 308ms/step - loss: 1.9628 - mae: 1.1542 - val_loss: 1.8690 - val_mae: 1.1432 - learning_rate: 1.0000e-03
Epoch 8/300
23/23 - 7s - 325ms/step - loss: 1.8854 - mae: 1.1364 - val_loss: 1.8053 - val

2025-08-23 13:19:48,586 - Model compiled and fitted.
2025-08-23 13:19:48,775 - Saved model to c:\Users\ashin\Documents\Dev\velib_ds\local\models\best.keras
2025-08-23 13:19:49,007 - Saved MLFlow run sneaky-hawk-555
2025-08-23 13:19:49,008 - All saved.


🏃 View run sneaky-hawk-555 at: http://127.0.0.1:8080/#/experiments/1/runs/868107c4b04b46faa5f3525ab8de730d
🧪 View experiment at: http://127.0.0.1:8080/#/experiments/1


In [4]:
df_transformed.head()


,cluster,pont,preholiday,vacances_uni,holiday,postholiday,delta,gel,lon,station,...,vent,lagged_value,lagged_value_3hours_mean,mean_value,mean_6_12,mean_16_22,sin_hour,sin_weekday,cos_hour,cos_weekday
33718,11,0,0,0,0,0,-5.000000,0.0,0.478902,1002059045,...,0.311111,0.486123,0.486809,0.497668,0.513966,0.37309,-0.258819,0.974928,0.965926,-0.222521
33719,24,0,0,0,0,0,0.333333,0.0,0.448016,100227241,...,0.311111,0.499579,0.499439,0.497668,0.513966,0.37309,-0.258819,0.974928,0.965926,-0.222521
33720,11,0,0,0,0,0,-6.666667,0.0,0.485784,1002738770,...,0.311111,0.481918,0.482599,0.497668,0.513966,0.37309,-0.258819,0.974928,0.965926,-0.222521
33721,11,0,0,0,0,0,1.333333,0.0,0.481615,100759356,...,0.311111,0.502103,0.500561,0.497668,0.513966,0.37309,-0.258819,0.974928,0.965926,-0.222521
33722,19,0,0,0,0,0,2.666667,0.0,0.512077,100760396,...,0.311111,0.505467,0.504210,0.497668,0.513966,0.37309,-0.258819,0.974928,0.965926,-0.222521
